# Large Mongolian News Language Model

Big thanks to [Florian Leuerer](https://github.com/floleuerer) for his ULMFiT notebooks for German ([repo link](https://github.com/floleuerer/fastai_ulmfit_german)). They helped significantly!

We will be using a large corpus of [Mongolian news articles](https://github.com/tugstugi/mongolian-nlp#datasets) (see online news corpus, 700 million words). 

Some basic cleaning was done to remove very short lines, and a 40% random sample was takent to reduce the total words to about 225 million.

## Library Installs and Imports

We will need several libraries to get started. Each should be installed and upgraded to ensure it works. This notebook was created in February 2020 and should work with the following major verisons:

- Fast.ai version 2.x
- Fastcore version: 1.x
- Pandas version: 1.x
- Numpy version: 1.x

In [7]:
!pip install -Uqq torchtext==0.8.0
!pip install -Uqq fastai
!pip install -Uqq fastcore
!pip install -Uqq pandas==1.1.0

     |████████████████████████████████| 7.0MB 7.8MB/s 
     |████████████████████████████████| 194kB 7.9MB/s 
     |████████████████████████████████| 12.8MB 10.8MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 10.5MB 8.0MB/s 


In [8]:
import fastai
import fastcore
from fastai.text.all import *
from fastai.callback.progress import CSVLogger

print('Fast.ai version:', fastai.__version__)
print('Fastcore version:', fastcore.__version__)

Fast.ai version: 2.2.7
Fastcore version: 1.3.19


In [9]:
import pandas as pd
import numpy as np

print('Pandas version:', pd.__version__)
print('numpy version:', np.__version__)

Pandas version: 1.1.5
numpy version: 1.19.5


For this project we will be using a Sentence Piece tokenizer. Fast.ai supports this, but we need to have the package installed first. 

In [10]:
!pip install -Uqq sentencepiece!=0.1.90,!=0.1.91

     |████████████████████████████████| 1.2MB 8.0MB/s 


In [ ]:
!pip install -Uqq gdown

In [ ]:
import gdown

We will be using Google Drive to store our files. Please change the directory for the `cd` command to wherever your notebook is located. 

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%cd '/content/drive/MyDrive/DataScience/NLP/02_mongolian_language_model'

/content/drive/MyDrive/DataScience/NLP/02_mongolian_language_model


## Download Data

In [ ]:
url = 'https://drive.google.com/uc?id=1oX4poQhCPlno2g131ohfQlSZ2VfMOKYt'
output = 'data/20_news.txt'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1oX4poQhCPlno2g131ohfQlSZ2VfMOKYt
To: /notebooks/Language Model/data/20_news.txt

0.00B [00:00, ?B/s]
524kB [00:00, 2.87MB/s]
1.57MB [00:00, 3.62MB/s]
3.67MB [00:00, 4.81MB/s]
6.29MB [00:00, 6.36MB/s]
7.86MB [00:00, 7.23MB/s]
13.1MB [00:00, 9.61MB/s]
15.7MB [00:00, 11.7MB/s]
18.4MB [00:01, 11.7MB/s]
22.5MB [00:01, 14.7MB/s]
25.2MB [00:01, 13.1MB/s]
29.4MB [00:01, 16.4MB/s]
32.0MB [00:01, 17.6MB/s]
34.6MB [00:01, 18.0MB/s]
37.7MB [00:01, 20.1MB/s]
40.4MB [00:02, 21.5MB/s]
43.0MB [00:02, 18.4MB/s]
47.2MB [00:02, 21.5MB/s]
50.3MB [00:02, 23.3MB/s]
53.5MB [00:02, 23.8MB/s]
56.1MB [00:02, 24.0MB/s]
58.7MB [00:02, 21.1MB/s]
62.4MB [00:02, 24.0MB/s]
65.6MB [00:03, 21.0MB/s]
69.2MB [00:03, 23.9MB/s]
72.4MB [00:03, 20.5MB/s]
76.6MB [00:03, 23.9MB/s]
79.5MB [00:03, 24.0MB/s]
82.7MB [00:03, 24.3MB/s]
85.8MB [00:04, 21.4MB/s]
89.7MB [00:04, 23.5MB/s]
92.3MB [00:04, 23.1MB/s]
95.1MB [00:04, 22.1MB/s]
98.0MB [00:04, 23.3MB/s]
101MB [00:04, 23.1MB/s

'data/20_news.txt'

## Create Data Loader for Language Model

We will be using data in a Pandas Dataframe to build our language model. However, you could also use txt files. 

In [ ]:
%ls data

20_news.txt


In [ ]:
df = pd.read_csv('data/20_news.txt', header=None, names=['text','len'])

In [ ]:
df.head()

,text,len
0,"Гэр хорооллын дахин төлөвлөлт хийхэд газраа өгсөн иргэд барилга ашиглалтад ортол амьдрах газар байдаггүй нь асуудал болдог. Гэр хорооллын дахин төлөвлөлтөд орсон айлуудыг газрыг нь чөлөөлүүлж авсан даруйдаа Түрээсийн орон сууцанд байлгаж болох талаар Барилга хот байгуулалтын сайд Ц.Баярсайхан Засгийн газрын ""11 11 төв""-д ажиллахдаа хэллээ. Түрээсийн орон сууцыг Голланд, Тайланд зэрэг орнуудад түгээмэл ашигладаг бөгөөд манай улсад гэр хорооллын дахин төлөвлөлтөд орсон айлуудад ашиглах бүрэн боломжтой аж. Тийм орон сууцыг зөвхөн төр хариуцахгүй бөгөөд хувийн хэвшилтэй хамтарч ашиглалтад оруу...",102
1,"хэргээр шалгана Улсын ерөнхий прокурор Д.Дорлигжав өчигдөр олны анхаарал татаж буй хоёр хэргийн талаар мэдээлэл өглөө. Тэрээр асуултад хариулах боломжгүй хэмээсэн учраас мэдээллийг товчлон хүргэе. 1. Алтанхүү гэдэг залууг хамтран амьдрагч эмэгтэйгээ зодож хөнөөсөн гээд хэвлэлээр бичсэн. Үүнийг тойроод маш олон мэдээлэл гарч, хүмүүсийн сэтгэл санааг түгшээгээд байна. Зарим нь оносон, нөгөө хэсэг нь ташаа мэдээлэл түгээж байна. Ерөнхийдөө энэ хоёр хүн хамтран амьдарч байсан. Хамтарч ууж идэж, наргиж явдаг байсан. Үүнийг нь найз нөхөд, үеэлүүд нь ч мэддэг, хэлдэг. Архи уусан үедээ эрэгтэй нь ...",830
2,"Тайланд улс сүүлийн хэдэн сарын турш дэлхийн уйл явдлын халуун цэгт тооцогдон бослого тэмцэл өрнөсөн энэ газар босогчдын гарт орж, дайны хөлд үрэгдэж, дарийн утаа ханхалсан айдас хүйдсээр дүүрэн гэсэн ойлголт олон хүнд төрсөн. Тиймээс нөхцөл байдал ямap байгааг газар дээрээс нь сурвалжлан хүргэх үүргийг манай сонин Уул уурхай, эдийн засгийн мэдээллийн албаны дарга П.Ядамдорж, гэрэл зургийн сурвалжлагч Б.Ганбаяр нартаа өгч алс холын Тайланд руу илгээсэн юм. Манай хоёр очих газраа сайн яваад хүрчээ. ""Хүннү эйр"" компанийн агаарын хөлгөөр зургаан цаг нисээд Бангкокт үүрээр буухад агаарын хэм +...",477
3,"Бүгд Найрамдах Түрк улсын Кайсери хотод суугаа Монгол Улсын Өргөмжит консул, ноён Хасан Хаскараманы санаачлагаар зохиолч Рыфат Сэртоглугийн бичсэн ""Чингис хааны Монгол"" ном түрк хэл дээр хэвлэгдэн гарлаа. Уг номыг дунд сургуулийн хүүхдүүдэд зориулж товч, энгийн хэллэгээр бичсэн бөгөөд Чингис хааны бага болон залуу насны амьдрал, тархай бутархай олон аймгийг нэгтгэн Их Монгол Улсыг байгуулан хаан ширээнд суусан түүх, түүний байлдан дагуулалт, цэргийн зохион байгуулалт, хүчин чадлын тухай өгүүлжээ. Мөн өнөөгийн Монгол Улс, нийслэл Улаанбаатар хотын эдийн засаг, нийгэм, соёл, боловсролын хөгж...",164
4,"Монголын оюутны спортын холбооноос энэ онд жүдо, гандбол, волейбол, шатар, ширээний теннис, тэквондо, сагсан бөмбөг, чөлөөт бөх, футзал, модон бөмбөг гэсэн спортын арван төрлөөр УАШТ-ийг зохион байгуулж байгаа билээ. Тэгвэл волейболын тэмцээн хоёрдугаар сарын 28-наас гуравдугаар сарын 3-нд МУИС-ийн Эдийн Засгийн сургуулийн спортын зааланд боллоо. Тэмцээнд нийслэл болон хөдөө орон нутгийн 13 их, дээд сургуулийн эрэгтэй 13, эмэгтэй 10 баг өрсөлдснөөс эрэгтэйчүүдээс Г.Галбадрах дасгалжуулагчтай М.Мөнхсайхан ахлагчтай МУИС-ийн баг аваргалж, спортын мастер С. Амгаабазар дасгалжуулагчтай Б.Наран...",136


In [ ]:
df = df.dropna()

In [ ]:
df.shape

(284159, 2)

We will be using the ColSplitter, which expects a column called is_valid (with a boolean) to generate the validation set. We will randomly assign 20% to the validation set.

In [ ]:
np.random.seed(42)

In [ ]:
df['is_valid'] = np.random.choice([0,1], size=len(df), p=[0.8, 0.2])

We can set our batch size with the `bs` argument.

The `is_lm` is set to `True` because we are training a language model. We are using a `get_x` but not a `get_y` because our target variable will be generated for us by the fastai library. 

In [ ]:
bs = 128

In [ ]:
tok = SentencePieceTokenizer(lang='mn', cache_dir='models/spm/lm')

In [ ]:
dblock = DataBlock(blocks=TextBlock.from_df('text', is_lm=True, tok=tok),
                   get_x=ColReader('text'),
                   splitter=ColSplitter())

dls = dblock.dataloaders(df, bs=bs, seq_len=80)

/opt/conda/envs/fastai/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Now you can see the `text_` column, which is our target variable. Our language model will be attempting to predict the next token in the sequence. 

In [ ]:
dls.show_batch()

,text,text_
0,"▁xxbos ▁сэлэнгэ ▁аймгийн ▁итх - ын ▁xxrep ▁3 ▁i ▁хуралдаан 11 - р ▁сарын ▁16- ны ▁өдөр ▁болж ▁89 ▁хувийн ▁ирц тэйгээр ▁хуралдлаа . ▁энэ ▁удаагийн ▁хуралдаанаар ▁аймгийн ▁засаг ▁даргын ▁2016-2020 ▁оны ▁үйл ▁ажиллагааны ▁хөтөлбөр , ▁байгалийн ▁ургамал ▁ашигласан ы ▁хэмжээг ▁шинэчлэн ▁тогтоох ▁тухай ▁төслийг ▁тус ▁бүр ▁хэлэлцлээ . ▁аймгийн ▁эдийн ▁засаг , ▁нийгмийн ▁хөгжлийн ▁хурдыг ▁ахиулах , ▁хүрсэн ▁үр ▁дүнг ▁бататган ▁хөгжлийн ▁шинэ ▁ололт , ▁амжилт уудаар ▁баяжуулах ыг ▁зорьж ▁боловсруулсан ▁аймгийн ▁засаг ▁даргын ▁үйл ▁ажиллагааны ▁хөтөлбөрийн ▁төслийг ▁""","▁сэлэнгэ ▁аймгийн ▁итх - ын ▁xxrep ▁3 ▁i ▁хуралдаан 11 - р ▁сарын ▁16- ны ▁өдөр ▁болж ▁89 ▁хувийн ▁ирц тэйгээр ▁хуралдлаа . ▁энэ ▁удаагийн ▁хуралдаанаар ▁аймгийн ▁засаг ▁даргын ▁2016-2020 ▁оны ▁үйл ▁ажиллагааны ▁хөтөлбөр , ▁байгалийн ▁ургамал ▁ашигласан ы ▁хэмжээг ▁шинэчлэн ▁тогтоох ▁тухай ▁төслийг ▁тус ▁бүр ▁хэлэлцлээ . ▁аймгийн ▁эдийн ▁засаг , ▁нийгмийн ▁хөгжлийн ▁хурдыг ▁ахиулах , ▁хүрсэн ▁үр ▁дүнг ▁бататган ▁хөгжлийн ▁шинэ ▁ололт , ▁амжилт уудаар ▁баяжуулах ыг ▁зорьж ▁боловсруулсан ▁аймгийн ▁засаг ▁даргын ▁үйл ▁ажиллагааны ▁хөтөлбөрийн ▁төслийг ▁"" монгол"
1,"▁байсан ▁... ▁/ ▁xxmaj ▁pa ul et te ▁xxmaj ▁k el ly ▁"" цэцэг "" ▁шүлэг ▁/ ▁онцлох , ▁олны ▁анхаарлын ▁төвд ▁байгаа ▁мэдээ ▁мэдээллийг ▁хүлээн ▁авахыг ▁хүсвэл ▁манай ▁фэйсбүүк ▁мессенжерт ▁холбогдоорой ! ▁ < !-- ▁xxbos ▁нийтэлсэн : ▁2014 - 05 -01 ▁13 :06 :13 ▁улсын ▁мөрдөн ▁байцаах ▁газраас ▁цаг ▁үеийн ▁асуудлаар ▁өчигдөр ▁мэдээлэл ▁хийсэн . ▁энэхүү ▁мэдээллийг ▁багц лан ▁хүргэе . ▁тухайлбал , ▁хар ▁тамхины ▁хэргээр ▁өнөөдрийн ▁байдлаар ▁монголын ▁таван ▁иргэн ▁цааз луулж ээ . ▁2002-200 9 ▁онд","▁... ▁/ ▁xxmaj ▁pa ul et te ▁xxmaj ▁k el ly ▁"" цэцэг "" ▁шүлэг ▁/ ▁онцлох , ▁олны ▁анхаарлын ▁төвд ▁байгаа ▁мэдээ ▁мэдээллийг ▁хүлээн ▁авахыг ▁хүсвэл ▁манай ▁фэйсбүүк ▁мессенжерт ▁холбогдоорой ! ▁ < !-- ▁xxbos ▁нийтэлсэн : ▁2014 - 05 -01 ▁13 :06 :13 ▁улсын ▁мөрдөн ▁байцаах ▁газраас ▁цаг ▁үеийн ▁асуудлаар ▁өчигдөр ▁мэдээлэл ▁хийсэн . ▁энэхүү ▁мэдээллийг ▁багц лан ▁хүргэе . ▁тухайлбал , ▁хар ▁тамхины ▁хэргээр ▁өнөөдрийн ▁байдлаар ▁монголын ▁таван ▁иргэн ▁цааз луулж ээ . ▁2002-200 9 ▁онд ▁буюу"
2,"▁ч ▁гаргасан ▁алдаа н ▁дээрээ ▁суралцсан ▁болов ▁уу . ▁гол ▁нь ▁сонгуулиар ▁бэлэн ▁мөнгө ▁амалж ▁болдоггүй н ▁гашуун ▁сургамж ийг ▁манай ▁улстөрчид ▁авсан ▁байгаасай ▁гэж ▁найд ъя . ▁сонгууль ▁болгон оор ▁намууд ▁бие , ▁биенээсээ ▁өрсөлдөж , ▁бэлэн ▁амалж ▁халамж ийг ▁хавтгай р уулсны ▁үр ▁дагавар т ▁монгол ▁хүний ▁хөдөлмөрийн ▁үнэлэмж ▁шал даа ▁буусан ▁гэж ▁хэлж ▁болно . ▁хөдөлмөрийн ▁үнэлэмж гүй ▁газарт ▁бүтээмж ийн ▁тухай ▁ярих ▁ч ▁хэрэггүй ▁юм . ▁хямрал ын ▁гурав ▁дахь ▁сайн ▁тал ▁нь ▁монголчууд ▁"" ажил","▁гаргасан ▁алдаа н ▁дээрээ ▁суралцсан ▁болов ▁уу . ▁гол ▁нь ▁сонгуулиар ▁бэлэн ▁мөнгө ▁амалж ▁болдоггүй н ▁гашуун ▁сургамж ийг ▁манай ▁улстөрчид ▁авсан ▁байгаасай ▁гэж ▁найд ъя . ▁сонгууль ▁болгон оор ▁намууд ▁бие , ▁биенээсээ ▁өрсөлдөж , ▁бэлэн ▁амалж ▁халамж ийг ▁хавтгай р уулсны ▁үр ▁дагавар т ▁монгол ▁хүний ▁хөдөлмөрийн ▁үнэлэмж ▁шал даа ▁буусан ▁гэж ▁хэлж ▁болно . ▁хөдөлмөрийн ▁үнэлэмж гүй ▁газарт ▁бүтээмж ийн ▁тухай ▁ярих ▁ч ▁хэрэггүй ▁юм . ▁хямрал ын ▁гурав ▁дахь ▁сайн ▁тал ▁нь ▁монголчууд ▁"" ажил ▁хийвэл"
3,"▁ажлыг ▁өнгөрсөн ▁жилийн ▁зургаадугаар ▁сард ▁эхэлсэн . ▁ол ле ▁гэдэг ▁нь ▁же жү гийн ▁эртний ▁ая л гаар ▁"" гудамж ▁талбайгаас ▁гэрийн ▁хашаа ▁хүртэлх ▁нарийхан ▁зам "" ▁гэсэн ▁утгатай ▁үг ▁юм . ▁бнсу - д ▁2006 ▁оноос ▁хойш ▁че жу ▁ арлыг ▁тойрсон ▁26 ▁курс , ▁4 25 ▁км ▁явган ▁аяллын ▁чиглэлийг ▁тэмдэг жүүлсэн ▁байдаг ▁аж . ▁ол лэ ▁явган ▁аяллын ▁замд ▁байгалийн ▁өнгө ▁үзэмж ийг ▁өөрчлөх ▁сандал , ▁амрах ▁цэгүүд ▁байхгүй . ▁ол ле ▁явган ▁аяллын ▁чиглэл ▁солонгост ▁хурдан","▁өнгөрсөн ▁жилийн ▁зургаадугаар ▁сард ▁эхэлсэн . ▁ол ле ▁гэдэг ▁нь ▁же жү гийн ▁эртний ▁ая л гаар ▁"" гудамж ▁талбайгаас ▁гэрийн ▁хашаа ▁хүртэлх ▁нарийхан ▁зам "" ▁гэсэн ▁утгатай ▁үг ▁юм . ▁бнсу - д ▁2006 ▁оноос ▁хойш ▁че жу ▁ арлыг ▁тойрсон ▁26 ▁курс , ▁4 25 ▁км ▁явган ▁аяллын ▁чиглэлийг ▁тэмдэг жүүлсэн ▁байдаг ▁аж . ▁ол лэ 

Our code below show the number of sequences of 80 tokens for our training and validation sets. 

In [ ]:
len(dls.train), len(dls.valid)

(11791, 2991)

Here we can move to the language model training, or if the training is already compplete, move to below to test the model.

## Training The Language Model

In [ ]:
notebook_path = Path('')

We will be using the AWD_LSTM architecture for our language model. We set the `pretrained` option to `False` as we will not be using a pretrained language model (that's what we are doing!).

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=False, 
                               metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.path = notebook_path.absolute()

Here we set our learning rate. Ideally we would find our ideal learning rate for our data. 

In [ ]:
lr = 1e-2
#lr *= bs/48  # Scale learning rate by batch size

In [ ]:
cbs=[CSVLogger(fname='history_20_news_lm.csv')]

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8, 0.7, 0.8), cbs=cbs)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.325104,4.293908,0.284503,73.252151,3:08:11
1,4.221550,4.173669,0.293306,64.953354,3:08:58
2,4.176488,4.060854,0.303038,58.023838,3:08:58
3,4.090224,3.967691,0.311900,52.862328,3:10:10
4,3.996021,3.879237,0.320767,48.387268,3:09:43
5,3.937507,3.783985,0.330628,43.991005,3:09:39
6,3.806522,3.678660,0.342809,39.593292,3:11:00
7,3.701015,3.577911,0.354890,35.798683,3:09:05
8,3.624425,3.507953,0.364043,33.379875,3:09:08
9,3.634460,3.488757,0.366848,32.745209,3:09:15


In [ ]:
learn.to_fp32().save('mn_20_news_lm', with_opt=False)

Path('/notebooks/Language Model/models/mn_20_news_lm.pth')

In [ ]:
with open('models/mn_20_news_vocab.pkl', 'wb') as f:
      pickle.dump(learn.dls.vocab, f)

In [ ]:
learn.save_encoder('mn_20_news_lm_encoder')

## Test out the model

In [13]:
lm_fns = ['models/mn_20_news_lm', 'models/mn_20_news_vocab']

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=True, 
                               pretrained_fnames=lm_fns, model_dir='', metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
TEXT = "Барилга угсралт их засварын ажил"

In [ ]:
pred = learn.predict(TEXT, 50, temperature=0.75)

In [ ]:
pred

'▁xxbos ▁барилга ▁угсралт ▁их ▁засварын ▁ажил ▁7 ▁дугаар ▁сарын ▁15- наас ▁эхэлнэ . ▁нийслэлийн ▁удирдах ▁ажилтнуудын ▁шуурхай ▁зөвлөлгөөнөөр ▁орон ▁сууц ▁нийтийн ▁аж ▁ахуйн ▁салбарын ▁удирдах ▁ажилтны ▁зөвлөгөөн ▁боллоо . ▁зөвлөгөөнд ▁нийслэлийн ▁мэргэжлийн ▁хяналтын ▁газрын ▁дарга ▁л . эрдэнэчулуун , ▁нийслэлийн ▁авто ▁замын ▁хөгжлийн ▁газрын ▁дарга ▁ж . тогтохбаяр , ▁нийслэлийн ▁замын ▁хөдөлгөөний ▁удирдлагын ▁газрын ▁дарга ▁с'

In [1]:
pred = '▁xxbos ▁барилга ▁угсралт ▁их ▁засварын ▁ажил ▁7 ▁дугаар ▁сарын ▁15- наас ▁эхэлнэ . ▁нийслэлийн ▁удирдах ▁ажилтнуудын ▁шуурхай ▁зөвлөлгөөнөөр ▁орон ▁сууц ▁нийтийн ▁аж ▁ахуйн ▁салбарын ▁удирдах ▁ажилтны ▁зөвлөгөөн ▁боллоо . ▁зөвлөгөөнд ▁нийслэлийн ▁мэргэжлийн ▁хяналтын ▁газрын ▁дарга ▁л . эрдэнэчулуун , ▁нийслэлийн ▁авто ▁замын ▁хөгжлийн ▁газрын ▁дарга ▁ж . тогтохбаяр , ▁нийслэлийн ▁замын ▁хөдөлгөөний ▁удирдлагын ▁газрын ▁дарга ▁с'

In [3]:
words = pred.split()

In [6]:
''.join(words).replace('▁', ' ')

' xxbos барилга угсралт их засварын ажил 7 дугаар сарын 15-наас эхэлнэ. нийслэлийн удирдах ажилтнуудын шуурхай зөвлөлгөөнөөр орон сууц нийтийн аж ахуйн салбарын удирдах ажилтны зөвлөгөөн боллоо. зөвлөгөөнд нийслэлийн мэргэжлийн хяналтын газрын дарга л.эрдэнэчулуун, нийслэлийн авто замын хөгжлийн газрын дарга ж.тогтохбаяр, нийслэлийн замын хөдөлгөөний удирдлагын газрын дарга с'